In [2]:
import nltk
import pymorphy2
import os
import codecs
import cPickle
import numpy as np
import time
import gensim

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import pairwise_distances as sk_dist

In [15]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Base initializations

In [4]:
# Folders and files
__DATA_FOLDER = os.path.join('data', 'news') # Folder with raw computerra corpus
__PICKLE_FOLDER = os.path.join('data', 'pickle') # Folder with serialied clear corpus
__CLEAR_CORPUS_FOLDER = os.path.join(__PICKLE_FOLDER, 'news') # Folder with serialied clear corpus
__MODELS_FOLDER = 'models'

# Normalize options
__NORMALIZE_CORPUS = 0 # option to run normalization of the corpus

# Random Seed
__RND_SEED = 1

# Tokenization options
morph = pymorphy2.MorphAnalyzer()
word_pattern = u'(?u)\w+'
tokenizer = RegexpTokenizer(word_pattern)
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Make dirs
if not os.path.exists(__PICKLE_FOLDER):
    os.mkdir(__PICKLE_FOLDER)
if not os.path.exists(__CLEAR_CORPUS_FOLDER):
    os.mkdir(__CLEAR_CORPUS_FOLDER)
if not os.path.exists(__MODELS_FOLDER):
    os.mkdir(__MODELS_FOLDER)

# Functions and classes

In [5]:
def normalize_text(text):
    '''
    Function clears text content with normalization
    
    Args:
    text - string with unnormalized content
    
    Returns:
    normalized_text - string with normalized content
    '''
    tokenized_text = []

    # sentence tokenizer
    raw_sentences = sentence_tokenizer.tokenize(text.strip())
    new_sentences = []
    
    for sentence in raw_sentences:
        new_sentence = ''
        for token in tokenizer.tokenize(sentence.strip()):
            if not token.isdigit():
                gram_info = morph.parse(token)
                new_sentence += ' ' + (gram_info[0].normal_form)
        if len(new_sentence):
            new_sentences.append(new_sentence.strip())
    
    normalized_text = '. '.join(new_sentences).strip()
    return normalized_text


def load_computerra_corpus(filepath):
    '''
    Function loads computerra document corpus
    Ignores .txt documents at 2nd level

    Agrs:
    filepath - path to corpus

    Returns:
    titles - list of documents titels
    docs - list of documents content
    '''

    titles = []
    docs = []

    for root, directories, filenames in os.walk(data_folder):       
        for filename in filenames: 
            _, file_extension = os.path.splitext(filename)
            if file_extension == '.txt':
                file_to_read = os.path.join(root, filename)
                with codecs.open(file_to_read,
                                 mode='rb',
                                 encoding='cp1251') as fin:
                    
                    content = fin.read()\
                                 .split('='*75)[2:-1]                       
                    content = '. '.join(content).strip()
                    
                    if content.count(' ') >= 10:                    
                        titles.append(file_to_read)
                        docs.append(content)
    return titles, docs


class LabeledLineSentence(object):
    '''
    Class for doc2vec
    '''
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter_old__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(doc.split(), [self.labels_list[idx]])
    def __iter__(self):
        for idx in np.random.choice(len(self.doc_list), size=len(self.doc_list), replace=False):
            yield LabeledSentence(self.doc_list[idx].split(), [self.labels_list[idx]])

# Load and normalize corpus

In [6]:
%%time

if __NORMALIZE_CORPUS or not os.path.exists(os.path.join(__CLEAR_CORPUS_FOLDER, 'docs_clear.p')):
    print 'Running corpus normalization'
    
    # Tokenization options
    morph = pymorphy2.MorphAnalyzer()
    word_pattern = u'(?u)\w+'
    tokenizer = RegexpTokenizer(word_pattern)
    sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    # Load original content
    titles, docs = load_computerra_corpus(__DATA_FOLDER)

    # Clear it
    docs_clear = map(normalize_text, docs)
    
    # Dump
    with open(os.path.join(__CLEAR_CORPUS_FOLDER, 'docs_clear.p'), 'wb') as fout:
        cPickle.dump(docs_clear, fout)
    with open(os.path.join(__CLEAR_CORPUS_FOLDER, 'docs.p'), 'wb') as fout:
        cPickle.dump(docs, fout)
    with open(os.path.join(__CLEAR_CORPUS_FOLDER, 'titles.p'), 'wb') as fout:
        cPickle.dump(titles, fout)
else:
    print 'Loading prenomalized corpus'
    
    # Load
    with open(os.path.join(__CLEAR_CORPUS_FOLDER, 'docs_clear.p'), 'rb') as fin:
        docs_clear = cPickle.load(fin)
    with open(os.path.join(__CLEAR_CORPUS_FOLDER, 'docs.p'), 'rb') as fin:
        docs = cPickle.load(fin)
    with open(os.path.join(__CLEAR_CORPUS_FOLDER, 'titles.p'), 'rb') as fin:
        titles = cPickle.load(fin)

Loading prenomalized corpus
Wall time: 26 s


# Run LSA

In [7]:
%%time

# stopwords modifications
stopwords_rus = stopwords.words('russian')

# TF-IDF vectorizer
vect = TfidfVectorizer(stop_words=stopwords_rus, 
                       binary=False,
                       ngram_range=(1,1),
                       norm='l2',
                       sublinear_tf=False,
                       min_df=5)

print 'Running vectorization'
X = vect.fit_transform(docs_clear)
print X.shape

# LSA
lsa = TruncatedSVD(n_components=400, algorithm='arpack')

print 'Running LSA'
Y = lsa.fit_transform(X)

Running vectorization
(32083, 29300)
Running LSA
Wall time: 1min 1s


# DEMO

## Similarity with documents from corpus

In [9]:
# Pick random document from corpus
i_doc = np.random.randint(Y.shape[0])
doc_id = titles[i_doc]
sims = sk_dist(Y[i_doc,:], Y=Y, metric='cosine').ravel()

sims_argsort = np.argsort(sims)

print u'Query (%s):\n «%s...»\n' % (doc_id, docs[i_doc][:1000]) 
print u'='*30
print u'TOP-3 similar documents %s:\n'


for label, index in [('top-1', 1), ('top-2', 2), ('top-3', 3)]:
    print(u'%s - %s, Similarity=%f:\n «%s...»\n' % (label,
                                                    titles[sims_argsort[index]],
                                                    1-sims[sims_argsort[index]],
                                                    docs[sims_argsort[index]][:1000]))

Query (data\news\080404week\newsDump\100681):
 «Заказчик убийства Сергея Юшенкова не хочет сидеть 20 лет
Адвокат Михаила Коданева, приговоренного к 20 годам лишения свободы по делу об убийстве депутата Сергея Юенкова, утверждает, что его подзхащитный пытался покончить жизнь самоубийством.
"Я распологаю сведениями, что Коданев пытался покончить жизнь самоубийством и был госпитализирован в Институт Склифосовского", - сказал РИА "Новости" адвокат Генри Резник.
Он не смог уточнить, когда именно Коданев предпринял попытку самоубийства:
"Это случилось в день оглашения приговора или накануне".
Резник также ничего не сказал о состоянии здоровья Коданева в настоящее время.
"Он переведен в медчасть Бутырского изолятора, сейчас к нему пытается пробиться еще один адвокат", - сказал адвокат.
Ранее источник в ГУИН Минюста сообщил РИА "Новости", что состояние здоровья Коданева ухудшилось накануне во второй половине дня, после чего он был помещен в медчасть.
О причинах ухудшения здоровья Коданева ниче

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


## Similarity with arbitrary document

In [14]:
file_to_read = 'data/news/shevardWeek/newsDump/100'

# Loading raw file content
with codecs.open(file_to_read,
                 mode='rb',
                 encoding='cp1251') as fin:
                    raw_content = fin.read().strip()

content = normalize_text(raw_content)
content_vect = vect.transform([content])
content_lsa = lsa.transform(content_vect)

sims = sk_dist(content_lsa, Y=Y, metric='cosine').ravel()

sims_argsort = np.argsort(sims)

print u'Query:\n «%s...»\n' % (raw_content[:1000]) 
print u'='*30
print u'TOP-3 similar documents %s:\n'


for label, index in [('top-1', 0), ('top-2', 1), ('top-3', 2)]:
    print(u'%s - %s, Similarity=%f:\n «%s...»\n' % (label,
                                                    titles[sims_argsort[index]],
                                                    1-sims[sims_argsort[index]],
                                                    docs[sims_argsort[index]][:1000]))

Query:
 «Счета международного аэропорта Толмачево арестованы
Сегодня служба судебных приставов отказала Толмачево в отложении сроков исполнительных действий по взысканию с аэропорта 102 млн. рублей в счет долга перед Российским авиационным консорциумом.
Об этом корреспонденту ИА REGNUM сообщил финансовый директор Сергей Ченчиков.
Приставы побывали в аэропорте, изъяли из кассы определенную сумму и наложили аресты на счета аэропорта, послав соответствующие предписания в банки о прекращении движения по счетам.
По словам гендиректора это обстоятельство практически дестабилизирует работу аэропорта.
Менеджер высказал беспокойство тем, что аэропорт лишится возможности нормального функционирования и обеспечения топливом и электроэнергией.
"Руководство аэропорта попытается добиться переносов срока исполнительных действий по взысканию долгов еще до того, как арест счетов скажется на работе порта", - заверил Сергей Ченчиков.
Напомним, что сегодня приставы выполнили августовское решение кассационн